In [1]:
import dask_ml.cluster

In [7]:
import glob

deck_names = glob.glob("processed_decks/*")
decks = []

In [8]:
for filename in deck_names:
    with open(filename,'r') as deck:
        decks.append(deck.read())

In [10]:
for i,deck in enumerate(decks):
    decks[i] = deck.split('\n')[1:]

In [13]:
for deck in decks:
    for i, card in enumerate(deck):
        quantity = int(card.split(' ')[0])
        card_name = ' '.join(card.split(' ')[1:])
        deck[i] = (quantity,card_name)

In [14]:
decks

[[(4, 'Ancient Ziggurat'),
  (4, 'Cavern of Souls'),
  (4, 'Horizon Canopy'),
  (1, 'Plains'),
  (2, 'Seachrome Coast'),
  (4, 'Unclaimed Territory'),
  (4, 'Champion of the Parish'),
  (1, 'Dark Confidant'),
  (1, 'Dire Fleet Daredevil'),
  (4, 'Kitesail Freebooter'),
  (4, 'Mantis Rider'),
  (4, 'Meddling Mage'),
  (1, 'Mirran Crusader'),
  (4, 'Noble Hierarch'),
  (3, 'Phantasmal Image'),
  (4, 'Reflector Mage'),
  (4, "Thalia\\'s Lieutenant"),
  (3, 'Thalia, Guardian of Thraben'),
  (4, 'Aether Vial')],
 [(3, 'Blackcleave Cliffs'),
  (1, 'Blood Crypt'),
  (3, 'Bloodstained Mire'),
  (2, 'Forest'),
  (1, 'Mountain'),
  (2, 'Overgrown Tomb'),
  (3, 'Raging Ravine'),
  (1, 'Stomping Ground'),
  (2, 'Swamp'),
  (1, 'Treetop Village'),
  (4, 'Verdant Catacombs'),
  (1, 'Wooded Foothills'),
  (2, 'Bloodbraid Elf'),
  (4, 'Dark Confidant'),
  (1, 'Grim Lavamancer'),
  (1, 'Kalitas, Traitor of Ghet'),
  (3, 'Scavenging Ooze'),
  (4, 'Tarmogoyf'),
  (1, 'Collective Brutality'),
  (3, 'Fatal

In [21]:
def card_names(a_deck):
    return [card[1] for card in a_deck]

In [25]:
all_card_names = []
for deck_card_names in [card_names(deck) for deck in decks]:
    all_card_names+=deck_card_names

In [27]:
len(all_card_names)

17052

In [29]:
all_card_names = set(all_card_names)

In [30]:
len(all_card_names)

642

In [31]:
 all_card_names = list(all_card_names) 

In [40]:
def deck_to_vector(deck):
    v = [0]*len(all_card_names)
    for i, name in enumerate(all_card_names):
        for number, card_name in deck:
            if card_name == name:
                v[i]+=number
    return v

In [42]:
deck_vectors = [deck_to_vector(deck) for deck in decks]

In [43]:
km = dask_ml.cluster.KMeans(n_clusters=8, oversampling_factor=5)
km.fit(deck_vectors)

KMeans(algorithm='full', copy_x=True, init='k-means||', init_max_iter=None,
    max_iter=300, n_clusters=8, n_jobs=1, oversampling_factor=5,
    precompute_distances='auto', random_state=None, tol=0.0001)

In [44]:
km

KMeans(algorithm='full', copy_x=True, init='k-means||', init_max_iter=None,
    max_iter=300, n_clusters=8, n_jobs=1, oversampling_factor=5,
    precompute_distances='auto', random_state=None, tol=0.0001)

In [51]:
labels = list(km.labels_.compute())

In [69]:
decks_labels = list(zip(decks,labels))

In [98]:
def most_common_cards(deck, k):
    deck.sort(key = lambda deck: deck[0], reverse=True)
    return [card[1] for card in deck[:k]]  

def decks_by_label(a_label):
    return [(deck, label) for (deck, label) in decks_labels if label == a_label]

In [99]:
k = 7
N = 10
for deck, label in decks_by_label(0)[:N]:
    print(str(most_common_cards(deck, k))+" "+ str(label))

['Ancient Ziggurat', 'Cavern of Souls', 'Horizon Canopy', 'Unclaimed Territory', 'Champion of the Parish', 'Kitesail Freebooter', 'Mantis Rider'] 0
['Windswept Heath', 'Flickerwisp', 'Knight of the Reliquary', 'Noble Hierarch', 'Tidehollow Sculler', 'Wasteland Strangler', 'Collected Company'] 0
['Ancient Ziggurat', 'Cavern of Souls', 'Horizon Canopy', 'Unclaimed Territory', 'Champion of the Parish', 'Kitesail Freebooter', 'Mantis Rider'] 0
['Ancient Ziggurat', 'Cavern of Souls', 'Horizon Canopy', 'Unclaimed Territory', 'Champion of the Parish', 'Kitesail Freebooter', 'Mantis Rider'] 0
['Windswept Heath', 'Flickerwisp', 'Knight of the Reliquary', 'Noble Hierarch', 'Tidehollow Sculler', 'Wasteland Strangler', 'Collected Company'] 0
['Ancient Ziggurat', 'Cavern of Souls', 'Horizon Canopy', 'Unclaimed Territory', 'Champion of the Parish', 'Kitesail Freebooter', 'Mantis Rider'] 0
['Gemstone Mine', 'Simic Growth Chamber', 'Primeval Titan', 'Sakura-Tribe Scout', 'Explore', 'Serum Visions', "S

In [ ]:
k = 7
N = 10
for deck, label in decks_by_label(0)[:N]:
    print(str(most_common_cards(deck, k))+" "+ str(label))

In [105]:
def apparition_ratio(a_card):
    label_count = [0]*8
    for deck, label in decks_labels:
        if a_card in [card_name for _,card_name in deck]:
            label_count[label]+=1
    total_apps = sum(label_count)
    return ([count/total_apps for count in label_count],total_apps)

In [125]:
import math 

def distance(x,y):
    dist = 0.0
    for i, elem in enumerate(x):
        dist+= (elem-y[i])*(elem-y[i])
    return math.sqrt(dist)

def closest_cards(a_card, k):
    this_card = apparition_ratio(a_card)[0]
    distances = []
    for name in all_card_names:
        dist = distance(apparition_ratio(name)[0],this_card)
        distances.append((name, dist))
    distances.sort(key = lambda x: x[1])
    distances = [(name, dist) for name, dist in distances if name!=a_card]
    return [name for name,_ in distances[:k] ]

In [134]:
closest_cards("Sundering Titan",10)

['Bottled Cloister',
 'Myr Retriever',
 'Pyxis of Pandemonium',
 'Surgical Extraction',
 'Reality Smasher',
 'Chromatic Sphere',
 'Ancient Stirrings',
 'Endless One',
 'Matter Reshaper',
 'Ratchet Bomb']

In [140]:
apparition_ratio("Ratchet Bomb")

([0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], 2)

In [143]:
def versatile_cards(k):
    variances = []
    for name in all_card_names:
        versatility = sum([1 if x > 0 else 0 for x in apparition_ratio(name)[0]])
        variances.append((name, versatility))
    variances.sort(key = lambda x: x[1], reverse=True)
    return [name for name,_ in variances[:k] ]

In [145]:
versatile_cards(20)

['Lightning Bolt',
 'Mountain',
 'Forest',
 'Simian Spirit Guide',
 'Island',
 'Stomping Ground',
 'Dismember',
 'Wooded Foothills',
 'Mutavault',
 'Misty Rainforest',
 'Swamp',
 'Bloodbraid Elf',
 'Field of Ruin',
 'Steam Vents',
 'Mox Opal',
 'Arid Mesa',
 'Breeding Pool',
 'Blood Moon',
 'Gemstone Mine',
 'Abrupt Decay']

In [147]:
apparition_ratio("Mox Opal")

([0.059322033898305086,
  0.0,
  0.01694915254237288,
  0.4915254237288136,
  0.0,
  0.4322033898305085,
  0.0,
  0.0],
 118)